# Data Preparation

The goal of this notebook is to build a dataset for the simplified Early Warning System (EWS) focused on graduation outcomes. 

We reviewed and explored the public datasets that we gathered. Then, we took the features that we felt had the best predictive value and also performed feature engineering. We then combined all the datasets into one dataset to be used for data exploration: `ews_dataset.pkl`.


## Filtering Strategy

Our focus was on public school level data. Therefore Charter Schools and DASS (Dashboard Alternative School Status) were not included in the dataset. For reporting category, we kept it at the total level for simplicity. 

**Aggregate Level:** "S". Focuses analysis on individual schools.  
**CharterSchool:** "No" or "N". Excludes charter schools to focus on traditional public high schools.   
**DASS:** "No" or "N". Removes alternative/continuation programs so graduation rates reflect typical comprehensive high schools.   
**ReportingCategory:** "TA". Keeps aggregate totals for each school (not broken down by subgroup) to simplify modeling.

#### Library imports

In [2]:
# import libraries
import importlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
import sys

from pathlib import Path

In [3]:
# import other libraries
from helper import (
    rpkl, # build_cdscode
    create_county_fr_geography,
    create_safety_connectedness_features,
)

# check if jcds library is installed
package_name = "jcds"

if importlib.util.find_spec(package_name) is None:
    print(f" '{package_name}' not found. Installing from Github... ")
    subprocess.check_call(
        [
            sys.executable,
            "-m",
            "pip",
            "install",
            "https://github.com/junclemente/jcds.git",
        ]
    )
else:
    print(f" '{package_name}' is already installed.")

from jcds import eda as jeda
from jcds import reports as jrep

 'jcds' is already installed.


#### Dataset locations

In [4]:
# main data folder path
data_folder = Path("../data")
raw_pickle = Path(data_folder / "raw_pickle")

## Public Schools and Districts

This dataset is the foundation for our school-level analysis. It provides unique identifiers and key structural attributes for every public school in California. 
Each record contains a 14-digit CDSCode which is the unique school identifier across all merged dataset.

[Data Dictionary: Public Schools and Districts](https://www.cde.ca.gov/ds/si/ds/fspubschls.asp)


### Selected Features 

- cdscode 
- county
- charter
- eilcode
- virtual
- magnet 
- yearroundyn
- latitude
- longitude
- multilingual

See the [Data Dictionary](../docs/data_dictionary.md) for detailed description of each feature. 


In [5]:
# import school data
df_schooldata = rpkl(raw_pickle, "raw_school_data.pkl")


ℹ️ 'cdscode' already exists — skipping creation

📁 Columns in raw_school_data.pkl:
['cdscode', 'ncesdist', 'ncesschool', 'statustype', 'county', 'district', 'school', 'street', 'streetabr', 'city', 'zip', 'state', 'mailstreet', 'mailstrabr', 'mailcity', 'mailzip', 'mailstate', 'phone', 'ext', 'faxnumber', 'website', 'opendate', 'closeddate', 'charter', 'charternum', 'fundingtype', 'doc', 'doctype', 'soc', 'soctype', 'edopscode', 'edopsname', 'eilcode', 'eilname', 'gsoffered', 'gsserved', 'virtual', 'magnet', 'yearroundyn', 'federaldfcdistrictid', 'latitude', 'longitude', 'admfname', 'admlname', 'lastupdate', 'multilingual']


In [6]:
# columns retained from dataset
cols_schooldata = [
 'cdscode',
#  'ncesdist',
#  'ncesschool',
#  'statustype',
 'county',
#  'district',
#  'school',
#  'street',
#  'streetabr',
#  'city',
#  'zip',
#  'state',
#  'mailstreet',
#  'mailstrabr',
#  'mailcity',
#  'mailzip',
#  'mailstate',
#  'phone',
#  'ext',
#  'faxnumber',
#  'website',
#  'opendate',
#  'closeddate',
 'charter',
#  'charternum',
#  'fundingtype',
#  'doc',
#  'doctype',
#  'soc',
#  'soctype',
#  'edopscode',
#  'edopsname',
 'eilcode',
#  'eilname',
#  'gsoffered',
#  'gsserved',
 'virtual',
 'magnet',
 'yearroundyn',
#  'federaldfcdistrictid',
 'latitude',
 'longitude',
#  'admfname',
#  'admlname',
#  'lastupdate',
 'multilingual']


df_schooldata = df_schooldata[cols_schooldata]
df_schooldata.head()

,cdscode,county,charter,eilcode,virtual,magnet,yearroundyn,latitude,longitude,multilingual
59,01611190130229,Alameda,N,HS,N,N,N,37.764958,-122.24593,N
91,01611270130450,Alameda,N,HS,N,N,N,37.896661,-122.29257,N
113,01611430131177,Alameda,N,HS,N,N,N,37.868913,-122.27120,Y
153,01611500132225,Alameda,N,HS,N,N,N,37.705184,-122.07847,N
154,01611500133876,Alameda,N,HS,V,N,N,37.713501,-122.09222,N


In [7]:
# check if list has uniuqe cdscode
df_schooldata["cdscode"].is_unique
df_schooldata.head()

,cdscode,county,charter,eilcode,virtual,magnet,yearroundyn,latitude,longitude,multilingual
59,01611190130229,Alameda,N,HS,N,N,N,37.764958,-122.24593,N
91,01611270130450,Alameda,N,HS,N,N,N,37.896661,-122.29257,N
113,01611430131177,Alameda,N,HS,N,N,N,37.868913,-122.27120,Y
153,01611500132225,Alameda,N,HS,N,N,N,37.705184,-122.07847,N
154,01611500133876,Alameda,N,HS,V,N,N,37.713501,-122.09222,N


# California: Department of Education

## Adjusted Cohort Graduation Rate and Outcome Data (ACGR)

The ACGR dataset reports four-year graduation outcomes for each California public high school. It includes the percentage of students who graduate on time, drop out, or remain enrolled after four years, along with counts of students meeting UC/CSU requirements or earning special distinctions such as the Seal of Biliteracy. This dataset defines the primary target variable for the EWS model and provides key indicators of student progression and completion.

[Data Dictionary: ACGR](https://www.cde.ca.gov/ds/ad/fsacgr.asp)

### Selected Features
- cdscode
- cohortstudents
- regular_hs_diploma_graduates_rate
- met_uccsu_grad_reqs_rate
- seal_of_biliteracy_rate
- dropout_rate
- still_enrolled_rate

See the [Data Dictionary](../docs/data_dictionary.md) for detailed description of each feature. 

In [8]:
df_acgr = rpkl(raw_pickle, "raw_acgr.pkl")

✅ Added 'cdscode' using: countycode, districtcode, schoolcode

📁 Columns in raw_acgr.pkl:
['academicyear', 'aggregatelevel', 'countycode', 'districtcode', 'schoolcode', 'countyname', 'districtname', 'schoolname', 'charterschool', 'dass', 'reportingcategory', 'cohortstudents', 'regular_hs_diploma_graduates_count', 'regular_hs_diploma_graduates_rate', 'met_uccsu_grad_reqs_count', 'met_uccsu_grad_reqs_rate', 'seal_of_biliteracy_count', 'seal_of_biliteracy_rate', 'golden_state_seal_merit_diploma_count', 'golden_state_seal_merit_diploma_rate', 'chspe_completer_count', 'chspe_completer_rate', 'adult_ed_hs_diploma_count', 'adult_ed_hs_diploma_rate', 'sped_certificate_count', 'sped_certificate_rate', 'ged_completer_count', 'ged_completer_rate', 'other_transfer_count', 'other_transfer_rate', 'dropout_count', 'dropout_rate', 'still_enrolled_count', 'still_enrolled_rate', 'cdscode']


In [9]:
# columns retained from dataset
cols_acgr = [
 'cdscode',
#  'academicyear',
#  'aggregatelevel',
#  'countycode',
#  'districtcode',
#  'schoolcode',
#  'countyname',
#  'districtname',
#  'schoolname',
#  'charterschool',
#  'dass',
#  'reportingcategory',
 'cohortstudents', # QA for weighing
#  'regular_hs_diploma_graduates_count',
 'regular_hs_diploma_graduates_rate', # target variable
#  'met_uccsu_grad_reqs_count',
 'met_uccsu_grad_reqs_rate',
#  'seal_of_biliteracy_count',
 'seal_of_biliteracy_rate', # language proficiency ???
#  'golden_state_seal_merit_diploma_count',
#  'golden_state_seal_merit_diploma_rate',
#  'chspe_completer_count',
#  'chspe_completer_rate',
#  'adult_ed_hs_diploma_count',
#  'adult_ed_hs_diploma_rate',
#  'sped_certificate_count',
#  'sped_certificate_rate',
#  'ged_completer_count',
#  'ged_completer_rate',
#  'other_transfer_count',
#  'other_transfer_rate',
#  'dropout_count',
 'dropout_rate', # secondary target
#  'still_enrolled_count',
 'still_enrolled_rate' # 5th year senior
 ]


df_acgr = df_acgr[cols_acgr]
df_acgr.head()

,cdscode,cohortstudents,regular_hs_diploma_graduates_rate,met_uccsu_grad_reqs_rate,seal_of_biliteracy_rate,dropout_rate,still_enrolled_rate
66594,01316090131755,11,0.0,0.0,0.0,63.6,0.0
66654,01316170131763,38,63.2,0.0,33.3,2.6,28.9
66718,01611190000001,*,*,*,*,*,*
66782,01611190106401,43,100.0,95.3,2.3,0.0,0.0
66910,01611190130229,394,92.4,73.9,22.8,2.3,1.0


In [10]:
# checked cdscode is unique
df_acgr["cdscode"].is_unique

True

In [11]:
df_combined = df_schooldata.merge(
    df_acgr,
    on="cdscode",
    how="left"
)

df_combined.shape

(1067, 16)

## Absenteeism

### Chronic Absenteeism Data

The Chronic Absenteeism dataset provides school-level rates of students who were absent for 10% or more of instructional days during the academic year. This measure serves as a key attendance indicator within the EWS framework, reflecting patterns of disengagement that often precede academic risk or dropout.

[Data Dictionary: Chronic Absenteeism](https://www.cde.ca.gov/ds/ad/fsabd.asp)

#### Selected Features
- chronicabsenteeismrate
- cdscode

See the [Data Dictionary](../docs/data_dictionary.md) for detailed description of each feature. 

In [12]:
# load raw dataset and filter
df_chron_abs = rpkl(raw_pickle, "raw_chronic_absent.pkl")


✅ Added 'cdscode' using: county_code, district_code, school_code

📁 Columns in raw_chronic_absent.pkl:
['academic_year', 'aggregate_level', 'county_code', 'district_code', 'school_code', 'county_name', 'district_name', 'school_name', 'charter_school', 'reporting_category', 'chronicabsenteeismeligiblecumula', 'chronicabsenteeismcount', 'chronicabsenteeismrate', 'cdscode']


In [13]:
# select columns
chron_abs_cols = [
#  'academic_year',
#  'aggregate_level',
#  'county_code',
#  'district_code',
#  'school_code',
#  'county_name',
#  'district_name',
#  'school_name',
#  'charter_school',
#  'reporting_category',
#  'chronicabsenteeismeligiblecumula',
#  'chronicabsenteeismcount',
 'chronicabsenteeismrate',
 'cdscode']

df_chron_abs = df_chron_abs[chron_abs_cols]
df_chron_abs.head()

,chronicabsenteeismrate,cdscode
57598,84.4,01100170130419
57599,61.7,01100170130401
57621,8.8,01316090131755
57644,11.6,01316170131763
58027,3.7,01611196090013


In [14]:
df_combined = df_combined.merge(
    df_chron_abs,
    on="cdscode", 
    how="left"
)

df_combined.shape

(1067, 17)

### Absenteeism by Reason

The Absenteeism by Reason dataset breaks down total absences by category—such as unexcused absences and out-of-school suspensions—to capture underlying behavioral and disciplinary patterns. These variables complement the chronic absenteeism rate by offering insight into the causes of missed instructional time and broader indicators of student engagement.  

[Data Dictionary: Absenteeism by Reason](https://www.cde.ca.gov/ds/ad/fsabr.asp)

#### Selected Features
- elegible_cumulative_enrollment
- unexcuseed_absences_percent
- outofschool_suspension_absences_percent
- cdscode

See the [Data Dictionary](../docs/data_dictionary.md) for detailed description of each feature. 

In [15]:
df_abs = rpkl(raw_pickle, "raw_absent_reason.pkl")


✅ Added 'cdscode' using: county_code, district_code, school_code

📁 Columns in raw_absent_reason.pkl:
['academic_year', 'aggregate_level', 'county_code', 'district_code', 'school_code', 'county_name', 'district_name', 'school_name', 'charter_school', 'dass', 'reporting_category', 'eligible_cumulative_enrollment', 'count_of_students_with_one_or_more_absences', 'average_days_absent', 'total_days_absent', 'excused_absences_percent', 'unexcused_absences_percent', 'outofschool_suspension_absences_percent', 'incomplete_independent_study_absences_percent', 'excused_absences_count', 'unexcused_absences_count', 'outofschool_suspension_absences_count', 'incomplete_independent_study_absences_count', 'cdscode']


In [16]:
# columns retained from dataset
abs_cols = [
#  'academic_year',
#  'aggregate_level',
#  'county_code',
#  'district_code',
#  'school_code',
#  'county_name',
#  'district_name',
#  'school_name',
#  'charter_school',
#  'dass',
#  'reporting_category',
 'eligible_cumulative_enrollment',
#  'count_of_students_with_one_or_more_absences',
#  'average_days_absent',
#  'total_days_absent',
#  'excused_absences_percent',
 'unexcused_absences_percent',
 'outofschool_suspension_absences_percent',
#  'incomplete_independent_study_absences_percent',
#  'excused_absences_count',
#  'unexcused_absences_count',
#  'outofschool_suspension_absences_count',
#  'incomplete_independent_study_absences_count',
 'cdscode']


df_abs = df_abs[abs_cols]
df_abs.head()

,eligible_cumulative_enrollment,unexcused_absences_percent,outofschool_suspension_absences_percent,cdscode
583,67,46.9,0,01316090131755
608,329,41.8,1.8,01316170131763
628,22,0,0,01611190000000
647,170,20.2,0,01611190106401
670,473,28.4,0.2,01611190111765


In [17]:
df_combined = df_combined.merge(
    df_abs,
    on="cdscode", 
    how="left"
)

df_combined.shape

(1067, 20)

## Free or Reduced-Price Meal (Student Poverty)

The Free or Reduced-Price Meal (FRPM) dataset measures the percentage and count of students eligible for meal assistance under the National School Lunch Program. This serves as a proxy for socioeconomic disadvantage at the school level, providing crucial context for understanding equity gaps and their relationship to graduation outcomes within the EWS model.

[Data Dictionary: FRPM ](https://www.cde.ca.gov/ds/ad/fsspfrpm.asp)

### Selected Features
- percent__eligible_free_k12
- frpm_count_k12
- calpads_fall_1_certification_status
- cdscode

See the [Data Dictionary](../docs/data_dictionary.md) for detailed description of each feature.  

In [18]:
df_frpm = rpkl(raw_pickle, "raw_frpm.pkl")

✅ Added 'cdscode' using: county_code, district_code, school_code

📁 Columns in raw_frpm.pkl:
['academic_year', 'county_code', 'district_code', 'school_code', 'county_name', 'district_name', 'school_name', 'district_type', 'school_type', 'educational_option_type', 'nslp_provision_status', 'charter_school_yn', 'charter_school_number', 'charter_funding_type', 'irc', 'low_grade', 'high_grade', 'enrollment_k12', 'free_meal_count_k12', 'percent__eligible_free_k12', 'frpm_count_k12', 'percent__eligible_frpm_k12', 'enrollment_ages_517', 'free_meal_count_ages_517', 'percent__eligible_free_ages_517', 'frpm_count_ages_517', 'percent__eligible_frpm_ages_517', 'calpads_fall_1_certification_status', 'cdscode']


In [19]:
cols_frpm = [
    # 'academic_year', 
    # 'county_code', 
    # 'district_code', 
    # 'school_code',
    # 'county_name', 
    # 'district_name', 
    # 'school_name', 
    # 'district_type', 
    # 'school_type', 
    # 'educational_option_type', 
    # 'nslp_provision_status', 
    # 'charter_school_yn', 
    # 'charter_school_number', 
    # 'charter_funding_type', 
    # 'irc', 
    # 'low_grade', 
    # 'high_grade', 
    # 'enrollment_k12', 
    # 'free_meal_count_k12', 
    'percent__eligible_free_k12', 
    'frpm_count_k12', 
    # 'percent__eligible_frpm_k12', 
    # 'enrollment_ages_517', 
    # 'free_meal_count_ages_517', 
    # 'percent__eligible_free_ages_517', 
    # 'frpm_count_ages_517', 
    # 'percent__eligible_frpm_ages_517', 
    'calpads_fall_1_certification_status', 
    'cdscode'
    ]


df_frpm = df_frpm[cols_frpm]
df_frpm.head()

,percent__eligible_free_k12,frpm_count_k12,calpads_fall_1_certification_status,cdscode
0,0.789474,47,Y,01100170130419
1,1.000000,64,Y,01100170130401
14,1.000000,62,Y,01316090131755
15,1.000000,318,Y,01316170131763
17,0.172013,327,Y,01611190130229


In [20]:
df_combined = df_combined.merge(
    df_frpm, 
    on="cdscode", 
    how="left"
)

df_combined.shape

(1067, 23)

## CBEDS Data about Schools & Districts

The CBEDS dataset contains administrative and categorical information about schools and districts, including organizational levels, reporting sections, and annual values. After review, this dataset was not included in the final EWS dataset due to redundancy with other sources that provide more detailed and interpretable staffing and structural information (e.g., Student–Staff Ratio, Staff Education, and Staff Experience).

[Data Dictionary: CBEDS](https://www.cde.ca.gov/ds/ad/fscbedsorab19.asp)


In [21]:
df_cbeds = rpkl(raw_pickle, "raw_cbeds.pkl")

ℹ️ 'cdscode' already exists — skipping creation

📁 Columns in raw_cbeds.pkl:
['cdscode', 'countyname', 'districtname', 'schoolname', 'description', 'level', 'section', 'rownumber', 'value', 'year']


In [22]:
col_cbeds = [
    'cdscode', 
    # 'countyname', 
    # 'districtname', 
    # 'schoolname', 
    # 'description', 
    'level', 
    'section', 
    'rownumber', 
    'value', 
    'year']

df_cbeds = df_cbeds[col_cbeds]
df_cbeds.head()

,cdscode,level,section,rownumber,value,year
18,01100170112607,S,B,4,True,2122
19,01100170112607,S,B,8,True,2122
20,01100170112607,S,C,1,0,2122
21,01100170112607,S,C,2,0,2122
22,01100170112607,S,D,1,True,2122


## Staff Data Files

The Staff Data Files include detailed information on school personnel, covering both student–staff ratios and staff qualifications such as education levels and years of experience. These datasets help quantify a school’s instructional capacity and professional expertise—factors closely linked to student achievement and school stability. Features derived from these files capture the balance of experienced versus novice teachers and the overall education level of staff, providing valuable context for graduation outcomes in the EWS model.

### Student / Staff Ratio

#### Selected Features
- school_grade_span
- stu_tch_ratio
- stu_adm_ratio
- stu_psv_ratio
- cdscode

[Data Dictionary: Student-Staff Ratio](https://www.cde.ca.gov/ds/ad/fsstrat.asp)

In [23]:
df_ss_ratio = rpkl(raw_pickle, "raw_student_staff_ratio.pkl")

✅ Added 'cdscode' using: county_code, district_code, school_code

📁 Columns in raw_student_staff_ratio.pkl:
['academic_year', 'aggregate_level', 'county_code', 'district_code', 'school_code', 'county_name', 'district_name', 'school_name', 'charter_school', 'dass', 'school_grade_span', 'total_enr_n', 'tch_fte_n', 'adm_fte_n', 'psv_fte_n', 'oth_fte_n', 'stu_tch_ratio', 'stu_adm_ratio', 'stu_psv_ratio', 'stu_oth_ratio', 'cdscode']


In [24]:
cols_ss_ratio = [
    # 'academic_year', 
    # 'aggregate_level', 
    # 'county_code', 
    # 'district_code', 
    # 'school_code', 
    # 'county_name', 
    # 'district_name', 
    # 'school_name', 
    # 'charter_school', 
    # 'dass', 
    'school_grade_span', 
    # 'total_enr_n', 
    # 'tch_fte_n', 
    # 'adm_fte_n', 
    # 'psv_fte_n', 
    # 'oth_fte_n', 
    'stu_tch_ratio', 
    'stu_adm_ratio', 
    'stu_psv_ratio', 
    # 'stu_oth_ratio', 
    'cdscode']



df_ss_ratio = df_ss_ratio[cols_ss_ratio]
df_ss_ratio = df_ss_ratio[df_ss_ratio["school_grade_span"] == "GS_K12"]
df_ss_ratio.head()

,school_grade_span,stu_tch_ratio,stu_adm_ratio,stu_psv_ratio,cdscode
556,GS_K12,*,*,*,01100170000000
571,GS_K12,4.8,12.4,3.9,01316090131755
572,GS_K12,*,*,*,01316170000000
573,GS_K12,4.4,40.3,159,01316170131763
574,GS_K12,*,*,*,01611190000000


In [25]:
df_combined = df_combined.merge(
    df_ss_ratio, 
    on="cdscode",
    how="left"
)

df_combined.shape

(1067, 27)

### Staff Education

The Staff Education dataset details the distribution of school staff by highest degree earned, including associate, bachelor’s, master’s, and doctoral levels. From this dataset, degree counts were normalized by total staff to create percentage-based indicators representing the overall qualification profile of each school’s workforce. These features provide insight into teacher expertise and professional capacity, which are often correlated with student performance and graduation outcomes.

[Data Dictionary: Staff Education](https://www.cde.ca.gov/ds/ad/fssted.asp)

#### Selected Engineered Features
The raw Staff Education dataset included detailed counts of staff by degree type and several filtering attributes. After cleaning and normalization, only percentage-based indicators were retained for the final dataset to represent the relative educational composition of each school's staff. 

- cdscode
- pct_associate
- pct_bachelors
- pct_bachelors_plus
- pct_master
- pct_master_plus
- pct_doctorate
- pct_juris_doctor
- pct_no_degree



In [26]:
df_staff_ed = rpkl(raw_pickle, "raw_staff_edu.pkl")


✅ Added 'cdscode' using: county_code, district_code, school_code

📁 Columns in raw_staff_edu.pkl:
['academic_year', 'aggregate_level', 'county_code', 'district_code', 'school_code', 'county_name', 'district_name', 'school_name', 'charter_school', 'dass', 'staff_type', 'school_grade_span', 'staff_gender', 'total_staff_count', 'associate', 'baccalaureate', 'baccalaureate_plus', 'master', 'master_plus', 'doctorate', 'special_juris_doctor', 'none', 'cdscode']


In [27]:
cols_staff_ed = [
    # 'academic_year', 
    'aggregate_level', 
    # 'county_code', 
    # 'district_code', 
    # 'school_code', 
    # 'county_name', 
    # 'district_name', 
    # 'school_name', 
    'charter_school', 
    'dass', 
    'staff_type', 
    'school_grade_span', 
    'staff_gender', 
    'total_staff_count', 
    'associate', 
    'baccalaureate', 
    'baccalaureate_plus', 
    'master', 
    'master_plus', 
    'doctorate', 
    'special_juris_doctor', 
    'none', 
    'cdscode']


df_staff_ed = df_staff_ed[cols_staff_ed]
df_staff_ed = df_staff_ed[
  (df_staff_ed["school_grade_span"] == "GS_912") &
    (df_staff_ed["staff_type"] == "ALL") &
    (df_staff_ed["staff_gender"] == "ALL")
]

df_staff_ed.head()

,aggregate_level,charter_school,dass,staff_type,school_grade_span,staff_gender,total_staff_count,associate,baccalaureate,baccalaureate_plus,master,master_plus,doctorate,special_juris_doctor,none,cdscode
7491,S,N,N,ALL,GS_912,ALL,95,0,12,30,12,33,2,0,6,01611190130229
7503,S,N,N,ALL,GS_912,ALL,10,0,0,5,0,5,0,0,0,01611190106401
7723,S,N,N,ALL,GS_912,ALL,236,0,101,35,85,11,4,0,0,01611430131177
7998,S,N,N,ALL,GS_912,ALL,149,0,28,47,24,47,3,0,0,01611500132225
8116,S,N,N,ALL,GS_912,ALL,5,0,0,2,3,0,0,0,0,01611500130047


In [28]:
# normalize staff education

cols = ['total_staff_count', 
    'associate', 
    'baccalaureate', 
    'baccalaureate_plus', 
    'master', 
    'master_plus', 
    'doctorate', 
    'special_juris_doctor', 
    'none',]

# ensure columns are numeric
# df_staff_ed[cols] = df_staff_ed[cols].apply(pd.to_numeric, errors = "coerce")

def get_pct(df, numerator, denominator="total_staff_count"):
    num = pd.to_numeric(df[numerator], errors="coerce")
    den = pd.to_numeric(df[denominator], errors="coerce")

    denom = den.replace(0, pd.NA) # prevents division by zero
    pct = num / denom 
    return pct

df_staff_ed["pct_associate"] = get_pct(df_staff_ed, "associate")
df_staff_ed["pct_bachelors"] = get_pct(df_staff_ed, "baccalaureate")
df_staff_ed["pct_bachelors_plus"] = get_pct(df_staff_ed, "baccalaureate_plus")
df_staff_ed["pct_master"] = get_pct(df_staff_ed, "master")
df_staff_ed["pct_master_plus"] = get_pct(df_staff_ed, "master_plus")
df_staff_ed["pct_doctorate"] = get_pct(df_staff_ed, "doctorate")
df_staff_ed["pct_juris_doctor"] = get_pct(df_staff_ed, "special_juris_doctor")
df_staff_ed["pct_no_degree"] = get_pct(df_staff_ed, "none")


df_staff_ed.head()

,aggregate_level,charter_school,dass,staff_type,school_grade_span,staff_gender,total_staff_count,associate,baccalaureate,baccalaureate_plus,...,none,cdscode,pct_associate,pct_bachelors,pct_bachelors_plus,pct_master,pct_master_plus,pct_doctorate,pct_juris_doctor,pct_no_degree
7491,S,N,N,ALL,GS_912,ALL,95,0,12,30,...,6,01611190130229,0.0,0.126316,0.315789,0.126316,0.347368,0.021053,0.0,0.063158
7503,S,N,N,ALL,GS_912,ALL,10,0,0,5,...,0,01611190106401,0.0,0.000000,0.500000,0.000000,0.500000,0.000000,0.0,0.000000
7723,S,N,N,ALL,GS_912,ALL,236,0,101,35,...,0,01611430131177,0.0,0.427966,0.148305,0.360169,0.046610,0.016949,0.0,0.000000
7998,S,N,N,ALL,GS_912,ALL,149,0,28,47,...,0,01611500132225,0.0,0.187919,0.315436,0.161074,0.315436,0.020134,0.0,0.000000
8116,S,N,N,ALL,GS_912,ALL,5,0,0,2,...,0,01611500130047,0.0,0.000000,0.400000,0.600000,0.000000,0.000000,0.0,0.000000


In [29]:
staff_ed_cols = [
 'cdscode',
 'pct_associate',
 'pct_bachelors',
 'pct_bachelors_plus',
 'pct_master',
 'pct_master_plus',
 'pct_doctorate',
 'pct_juris_doctor',
 'pct_no_degree']

staff_ed = df_staff_ed[staff_ed_cols]
staff_ed.head()

,cdscode,pct_associate,pct_bachelors,pct_bachelors_plus,pct_master,pct_master_plus,pct_doctorate,pct_juris_doctor,pct_no_degree
7491,01611190130229,0.0,0.126316,0.315789,0.126316,0.347368,0.021053,0.0,0.063158
7503,01611190106401,0.0,0.000000,0.500000,0.000000,0.500000,0.000000,0.0,0.000000
7723,01611430131177,0.0,0.427966,0.148305,0.360169,0.046610,0.016949,0.0,0.000000
7998,01611500132225,0.0,0.187919,0.315436,0.161074,0.315436,0.020134,0.0,0.000000
8116,01611500130047,0.0,0.000000,0.400000,0.600000,0.000000,0.000000,0.0,0.000000


In [30]:
df_combined = df_combined.merge(
    staff_ed, 
    on="cdscode",
    how="left"
)

df_combined.shape

(1067, 35)

### Staff Experience

The Staff Experience dataset summarizes the composition of school staff by years of service and averages of total and district-level experience. It provides insight into faculty stability and institutional knowledge within each school. From this dataset, percentage-based indicators were derived to represent the proportion of experienced, inexperienced, first-year, and second-year teachers, offering valuable context for understanding how staff experience relates to student outcomes and graduation rates.

[Data Dictionary: Staff Experience](https://www.cde.ca.gov/ds/ad/fsstex.asp)

#### Selected Engineered Features
The raw Staff Experience dataset included detailed counts of staff by years of service, along with averages of total and district experience. After cleaning and transformation, only percentage-based indicators were retained for the final dataset to represent the relative distribution of experienced, inexperienced, first-year, and second-year staff within each school.

- cdscode
- pct_experienced
- pct_inexperienced
- pct_first_year
- pct_second_year

In [31]:
df_staff_xp = rpkl(raw_pickle, "raw_staff_exp.pkl")


✅ Added 'cdscode' using: county_code, district_code, school_code

📁 Columns in raw_staff_exp.pkl:
['academic_year', 'aggregate_level', 'county_code', 'district_code', 'school_code', 'county_name', 'district_name', 'school_name', 'charter_school', 'dass', 'staff_type', 'school_grade_span', 'staff_gender', 'total_staff_count', 'average_total_years_experience', 'average_district_years_experience', 'experienced', 'inexperienced', 'first_year', 'second_year', 'cdscode']


In [32]:
cols_staff_xp = [
    # 'academic_year', 
    # 'aggregate_level', 
    # 'county_code', 
    # 'district_code', 
    # 'school_code', 
    # 'county_name', 
    # 'district_name', 
    # 'school_name', 
    # 'charter_school', 
    # 'dass', 
    'staff_type', 
    'school_grade_span', 
    'staff_gender', 
    'total_staff_count', 
    'average_total_years_experience', 
    'average_district_years_experience', 
    'experienced', 
    'inexperienced', 
    'first_year', 
    'second_year', 
    'cdscode']

df_staff_xp = df_staff_xp[cols_staff_xp]
df_staff_xp = df_staff_xp[
    (df_staff_xp["staff_type"] == "ALL") &
    (df_staff_xp["staff_gender"] == "ALL") &
    (df_staff_xp["school_grade_span"] == "GS_912")
]

df_staff_xp.head()

,staff_type,school_grade_span,staff_gender,total_staff_count,average_total_years_experience,average_district_years_experience,experienced,inexperienced,first_year,second_year,cdscode
7491,ALL,GS_912,ALL,95,13.6,10.0,82,13,10,3,01611190130229
7503,ALL,GS_912,ALL,10,18.3,12.8,10,0,0,0,01611190106401
7723,ALL,GS_912,ALL,236,10.7,10.3,202,34,22,12,01611430131177
7998,ALL,GS_912,ALL,149,13.8,9.5,136,13,8,5,01611500132225
8116,ALL,GS_912,ALL,5,16.2,7.6,5,0,0,0,01611500130047


In [33]:
df_staff_xp["pct_experienced"] = get_pct(df_staff_xp, "experienced", "total_staff_count")
df_staff_xp["pct_inexperienced"] = get_pct(df_staff_xp, "inexperienced", "total_staff_count")
df_staff_xp["pct_first_year"] = get_pct(df_staff_xp, "first_year", "total_staff_count")
df_staff_xp["pct_second_year"] = get_pct(df_staff_xp, "second_year", "total_staff_count")

exp_cols = [
    "cdscode", 
    "pct_experienced", 
    "pct_inexperienced", 
    "pct_first_year", 
    "pct_second_year"
]

df_staff_experience = df_staff_xp[exp_cols].copy()
df_staff_experience.head()

,cdscode,pct_experienced,pct_inexperienced,pct_first_year,pct_second_year
7491,01611190130229,0.863158,0.136842,0.105263,0.031579
7503,01611190106401,1.000000,0.000000,0.000000,0.000000
7723,01611430131177,0.855932,0.144068,0.093220,0.050847
7998,01611500132225,0.912752,0.087248,0.053691,0.033557
8116,01611500130047,1.000000,0.000000,0.000000,0.000000


In [34]:
df_staff_experience["cdscode"].is_unique

True

In [35]:
df_combined = df_combined.merge(
    df_staff_experience, 
    on="cdscode", 
    how="left"
)

df_combined.shape

(1067, 39)

## Enrollment by School

The Enrollment by School dataset provides grade-level enrollment counts for each school, allowing analysis of cohort size and student progression across grades 9–12. Aggregated values were used to calculate ratios and percentages that capture grade retention and overall high school composition. These indicators help quantify student flow through the secondary grades and serve as proxies for cohort stability within the EWS model.

[Data Dictionary: Enrollment by School](https://www.cde.ca.gov/ds/ad/fsenrps.asp)

### Selected Engineered Features
The raw Enrollment by School dataset contained grade-level enrollment counts disaggregated by race, gender, and grade level. After aggregation and feature engineering, only ratio-based indicators were retained for the final dataset to capture grade progression and high school composition, reflecting student flow and cohort stability across grades 9–12.

- cdscode
- grade_retention_ratio
- pct_hs_enrollment
- pct_senior_cohort

See the [Data Dictionary](../docs/data_dictionary.md) for detailed description of each feature.  

In [36]:
df_enroll = rpkl(raw_pickle, "raw_school_enroll.pkl")


⚠️ Could not build 'cdscode' (missing county, district, school)

📁 Columns in raw_school_enroll.pkl:
['academic_year', 'cds_code', 'county', 'district', 'school', 'enr_type', 'race_ethnicity', 'gender', 'gr_kn', 'gr_1', 'gr_2', 'gr_3', 'gr_4', 'gr_5', 'gr_6', 'gr_7', 'gr_8', 'ungr_elm', 'gr_9', 'gr_10', 'gr_11', 'gr_12', 'ungr_sec', 'enr_total', 'adult']


In [37]:
df_enroll = jeda.rename_column(df_enroll, "cds_code", "cdscode")


In [38]:
cols_enroll = [
 'academic_year',
 'cdscode',
#  'county',
#  'district',
#  'school',
 'enr_type',
 'race_ethnicity',
 'gender',
#  'gr_kn',
#  'gr_1',
#  'gr_2',
#  'gr_3',
#  'gr_4',
#  'gr_5',
#  'gr_6',
#  'gr_7',
#  'gr_8',
#  'ungr_elm',
 'gr_9',
 'gr_10',
 'gr_11',
 'gr_12',
 'ungr_sec',
 'enr_total',
#  'adult'
 ]

df_enroll = df_enroll[cols_enroll]
df_enroll = df_enroll[df_enroll["academic_year"] == "2021-22"
    # (df_enroll["enr_type"] == "C")
]

df_enroll.head()

,academic_year,cdscode,enr_type,race_ethnicity,gender,gr_9,gr_10,gr_11,gr_12,ungr_sec,enr_total
262983,2021-22,01100170112607,C,0,F,0,0,3,1,0,11
262984,2021-22,01100170112607,C,0,M,0,0,0,1,0,3
262985,2021-22,01100170112607,C,1,F,0,1,1,0,0,2
262986,2021-22,01100170112607,C,2,F,0,0,0,0,0,1
262987,2021-22,01100170112607,C,2,M,0,0,1,0,0,1


In [39]:
df_enroll.shape

(132691, 11)

In [40]:
cols_to_agg = [ 
 'gr_9',
 'gr_10',
 'gr_11',
 'gr_12',
 'ungr_sec',
 'enr_total'
 ]

df_enroll[cols_to_agg] = (
    df_enroll[cols_to_agg].apply(pd.to_numeric, errors="coerce").fillna(0)
)


In [41]:
df_enroll_grouped = (
    df_enroll
    .groupby("cdscode", as_index = False)
    .agg({col: "sum" for col in cols_to_agg})
)

df_enroll_grouped["grade_retention_ratio"] = (
    df_enroll_grouped["gr_12"] / df_enroll_grouped["gr_9"].replace(0, np.nan)
)

df_enroll_grouped["pct_hs_enrollment"] = (
    (
        df_enroll_grouped["gr_9"] + 
        df_enroll_grouped["gr_10"] + 
        df_enroll_grouped["gr_11"] + 
        df_enroll_grouped["gr_12"]
    ) / df_enroll_grouped["enr_total"].replace(0, np.nan)
)

df_enroll_grouped["pct_senior_cohort"] = (
    (df_enroll_grouped["gr_11"] + df_enroll_grouped["gr_12"]) / 
    df_enroll_grouped["enr_total"].replace(0, np.nan)
)

df_enroll_grouped.head()

,cdscode,gr_9,gr_10,gr_11,gr_12,ungr_sec,enr_total,grade_retention_ratio,pct_hs_enrollment,pct_senior_cohort
0,01100170112607,59,60,85,83,0,411,1.406780,0.698297,0.408759
1,01100170123968,0,0,0,0,0,202,NaN,0.000000,0.000000
2,01100170124172,0,0,0,0,0,672,NaN,0.000000,0.000000
3,01100170125567,0,0,0,0,0,349,NaN,0.000000,0.000000
4,01100170130401,26,14,16,5,0,64,0.192308,0.953125,0.328125


In [42]:
cols_enroll = [
    "cdscode",
    "grade_retention_ratio",
    "pct_hs_enrollment", 
    "pct_senior_cohort"
]

df_combined = df_combined.merge(
    df_enroll_grouped[cols_enroll],
    on="cdscode",
    how="left"
)

df_combined.shape

(1067, 42)

# Ca DOE School Climate, Health, and Learning Surveys

[Data Information](https://calschls.org/reports-data/query-calschls/)

## Perception of Safety by Grade Level

The Perception of Safety by Grade Level dataset summarizes student responses from the California School Climate, Health, and Learning Survey (CalSCHLS) regarding how safe they feel at school. Data were filtered to include 11th-grade responses at the county level to align with high school populations. After processing, aggregated percentages of students who felt safe, unsafe, or neutral were retained to represent overall perceptions of school safety within each county.

### Selected Engineered Features
The raw dataset from the California School Climate, Health, and Learning Survey (CalSCHLS) included multiple grade levels and safety categories for each county across California. Each record reported the percentage of students who felt very safe, safe, neither safe nor unsafe, unsafe, or very unsafe at school, with data available for several reporting periods and school types.

For this project, the data were filtered to include only 11th-grade responses at the county level to best represent high school students and maintain consistency with the school-level data used in the EWS model. After cleaning and aggregation, the dataset was simplified to include combined percentages of students who felt safe, unsafe, or neutral, providing a high-level measure of perceived school safety across counties.

- county
- pct_unsafe_gr11
- pct_safe_gr11
- pct_neutral_gr11


In [43]:
df_safety = rpkl(raw_pickle, "raw_safety_percept_grade.pkl")

⚠️ Could not build 'cdscode' (missing county, district, school)

📁 Columns in raw_safety_percept_grade.pkl:
['geography', 'geo_type', 'grade', 'very_safe_pct', 'safe_pct', 'neither_pct', 'unsafe_pct', 'very_unsafe_pct', 'years', 'level_of_safety_filter']


In [44]:
cols_safety = [
    'geography', 
    'geo_type', 
    'grade', 
    'very_safe_pct', 
    'safe_pct', 
    'neither_pct', 
    'unsafe_pct', 
    'very_unsafe_pct', 
    'years', 
    'level_of_safety_filter'
]

df_safety = df_safety[
    (df_safety["geo_type"].str.strip() == "County") & 
    (df_safety["grade"] == 11)
]

df_safety.head()

,geography,geo_type,grade,very_safe_pct,safe_pct,neither_pct,unsafe_pct,very_unsafe_pct,years,level_of_safety_filter
3,Alameda County,County,11,0.145,0.423,0.351,0.051,0.029,2017-2019,All
5,Amador County,County,11,0.165,0.433,0.328,0.052,0.022,2017-2019,All
7,Butte County,County,11,0.117,0.643,0.200,0.027,0.013,2017-2019,All
9,Calaveras County,County,11,0.117,0.416,0.371,0.059,0.038,2017-2019,All
11,Colusa County,County,11,0.143,0.481,0.314,0.017,0.045,2017-2019,All


In [45]:
# def create_county_fr_geography(df, column="geography"):
#     df["county"] = (
#         df[column]
#         .str.replace(" County", "", regex=False)
#         .str.strip()
#     )

#     return df 

# df_safety["county"] = (
#     df_safety["geography"]
#     .str.replace(" County", "", regex=False)
#     .str.strip()
# )

df_safety = create_county_fr_geography(df_safety)

df_safety["pct_unsafe_gr11"] = df_safety["unsafe_pct"] + df_safety["very_unsafe_pct"]
df_safety["pct_safe_gr11"] = df_safety["safe_pct"] + df_safety["very_safe_pct"]
df_safety["pct_neutral_gr11"] = df_safety["neither_pct"]

df_safety.head()

,geography,geo_type,grade,very_safe_pct,safe_pct,neither_pct,unsafe_pct,very_unsafe_pct,years,level_of_safety_filter,county,pct_unsafe_gr11,pct_safe_gr11,pct_neutral_gr11
3,Alameda County,County,11,0.145,0.423,0.351,0.051,0.029,2017-2019,All,Alameda,0.080,0.568,0.351
5,Amador County,County,11,0.165,0.433,0.328,0.052,0.022,2017-2019,All,Amador,0.074,0.598,0.328
7,Butte County,County,11,0.117,0.643,0.200,0.027,0.013,2017-2019,All,Butte,0.040,0.760,0.200
9,Calaveras County,County,11,0.117,0.416,0.371,0.059,0.038,2017-2019,All,Calaveras,0.097,0.533,0.371
11,Colusa County,County,11,0.143,0.481,0.314,0.017,0.045,2017-2019,All,Colusa,0.062,0.624,0.314


In [46]:
cols_safety = [
#  'geography',
#  'geo_type',
#  'grade',
#  'very_safe_pct',
#  'safe_pct',
#  'neither_pct',
#  'unsafe_pct',
#  'very_unsafe_pct',
#  'years',
#  'level_of_safety_filter',
 'county',
 'pct_unsafe_gr11',
 'pct_safe_gr11',
 'pct_neutral_gr11']

df_combined = df_combined.merge(
    df_safety[cols_safety],
    on="county",
    how="left"
)

df_combined.shape

(1067, 45)

## Perception of Safety by School Connectedness
The raw dataset from the California School Climate, Health, and Learning Survey (CalSCHLS) reported safety perceptions broken down by levels of school connectedness—High, Medium, and Low—for each county. Each record contained the proportion of students who felt very safe, safe, neither safe nor unsafe, unsafe, or very unsafe within each connectedness group.

# Selected Engineered Features
For this project, the dataset was cleaned and standardized by county, and engineered features were created to capture the relationship between connectedness and perceived safety. These include an average safety score, connectedness ratios, and a composite school climate index summarizing both engagement and safety. Together, these measures provide a contextual view of school climate factors that may influence graduation outcomes.

- county
- avg_safety_score
- high_conn
- low_conn
- conn_ratio
- school_climate_index


In [47]:
df_connected = rpkl(raw_pickle, "raw_safety_connect.pkl")


⚠️ Could not build 'cdscode' (missing county, district, school)

📁 Columns in raw_safety_connect.pkl:
['geography', 'connectedness', 'very_safe', 'safe', 'neither_safe_nor_unsafe', 'unsafe', 'very_unsafe', 'safety_positive']


In [48]:
df_connected = create_county_fr_geography(df_connected)

df_connected.columns.to_list()

['geography',
 'connectedness',
 'very_safe',
 'safe',
 'neither_safe_nor_unsafe',
 'unsafe',
 'very_unsafe',
 'safety_positive',
 'county']

In [49]:
df_connected = jeda.rename_column(df_connected, "neither_safe_nor_unsafe", "neither")
df_connected.head()

,geography,connectedness,very_safe,safe,neither,unsafe,very_unsafe,safety_positive,county
0,California,High,0.268,0.559,0.157,0.011,0.005,0.827,California
1,California,Medium,0.052,0.334,0.520,0.065,0.028,0.386,California
2,California,Low,0.069,0.111,0.428,0.196,0.196,0.180,California
3,Alameda County,High,0.268,0.582,0.138,0.009,0.004,0.850,Alameda
4,Alameda County,Medium,0.060,0.370,0.494,0.057,0.020,0.430,Alameda


In [50]:
df_connected = create_safety_connectedness_features(df_connected)

In [51]:
df_connected.columns.to_list()

['county',
 'avg_safety_score',
 'high_conn',
 'low_conn',
 'conn_ratio',
 'school_climate_index']

In [52]:
cols_connected = ['county',
 'avg_safety_score',
 'high_conn',
 'low_conn',
 'conn_ratio',
 'school_climate_index']

df_connected[cols_connected]

,county,avg_safety_score,high_conn,low_conn,conn_ratio,school_climate_index
0,Alameda,0.034230,0.333333,0.333333,0.999997,0.517115
1,Alpine,NaN,0.333333,0.333333,0.999997,NaN
2,Amador,0.033827,0.333333,0.333333,0.999997,0.516913
3,Butte,0.034853,0.333333,0.333333,0.999997,0.517427
4,Calaveras,0.033593,0.333333,0.333333,0.999997,0.516797
5,California,0.033497,0.333333,0.333333,0.999997,0.516748
6,Colusa,0.033677,0.333333,0.333333,0.999997,0.516838
7,Contra Costa,0.034490,0.333333,0.333333,0.999997,0.517245
8,Del Norte,0.031287,0.333333,0.333333,0.999997,0.515643
9,El Dorado,0.034083,0.333333,0.333333,0.999997,0.517042


In [53]:
df_combined = df_combined.merge(
    df_connected[cols_connected],
    on="county",
    how="left"
)

print(df_combined.shape)
df_combined.head()

(1067, 50)


,cdscode,county,charter,eilcode,virtual,magnet,yearroundyn,latitude,longitude,multilingual,...,pct_hs_enrollment,pct_senior_cohort,pct_unsafe_gr11,pct_safe_gr11,pct_neutral_gr11,avg_safety_score,high_conn,low_conn,conn_ratio,school_climate_index
0,01611190130229,Alameda,N,HS,N,N,N,37.764958,-122.24593,N,...,1.00000,0.498894,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115
1,01611270130450,Alameda,N,HS,N,N,N,37.896661,-122.29257,N,...,1.00000,0.501264,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115
2,01611430131177,Alameda,N,HS,N,N,N,37.868913,-122.27120,Y,...,1.00000,0.496276,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115
3,01611500132225,Alameda,N,HS,N,N,N,37.705184,-122.07847,N,...,1.00000,0.470174,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115
4,01611500133876,Alameda,N,HS,V,N,N,37.713501,-122.09222,N,...,0.39881,0.196429,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115


# Basic dataset info

In [54]:
jrep.data_info(df_combined, show_columns=True)


SHAPE:
There are 1067 rows and 50 columns (1.63 MB).

DUPLICATES:
There are 0 duplicated rows.

COLUMNS/VARIABLES:
Column dType Summary:
 * object: 25
 * float: 25
There are 25 numerical (int/float/bool) variables.
 * Columns: ['percent__eligible_free_k12', 'frpm_count_k12', 'pct_associate', 'pct_bachelors', 'pct_bachelors_plus', 'pct_master', 'pct_master_plus', 'pct_doctorate', 'pct_juris_doctor', 'pct_no_degree', 'pct_experienced', 'pct_inexperienced', 'pct_first_year', 'pct_second_year', 'grade_retention_ratio', 'pct_hs_enrollment', 'pct_senior_cohort', 'pct_unsafe_gr11', 'pct_safe_gr11', 'pct_neutral_gr11', 'avg_safety_score', 'high_conn', 'low_conn', 'conn_ratio', 'school_climate_index']
There are 25 categorical (nominal/ordinal) variables.
 * Columns: ['cdscode', 'county', 'charter', 'eilcode', 'virtual', 'magnet', 'yearroundyn', 'latitude', 'longitude', 'multilingual', 'cohortstudents', 'regular_hs_diploma_graduates_rate', 'met_uccsu_grad_reqs_rate', 'seal_of_biliteracy_rate', 

In [58]:
# list features and target
target = "regular_hs_diploma_graduates_rate"
num_cols = ['percent__eligible_free_k12', 'frpm_count_k12', 'pct_associate', 'pct_bachelors', 'pct_bachelors_plus', 'pct_master', 'pct_master_plus', 'pct_doctorate', 'pct_juris_doctor', 'pct_no_degree', 'pct_experienced', 'pct_inexperienced', 'pct_first_year', 'pct_second_year', 'grade_retention_ratio', 'pct_hs_enrollment', 'pct_senior_cohort', 'pct_unsafe_gr11', 'pct_safe_gr11', 'pct_neutral_gr11', 'avg_safety_score', 'high_conn', 'low_conn', 'conn_ratio', 'school_climate_index']

cat_cols = ['cdscode', 'county', 'charter', 'eilcode', 'virtual', 'magnet', 'yearroundyn', 'latitude', 'longitude', 'multilingual', 'cohortstudents', 'regular_hs_diploma_graduates_rate', 'met_uccsu_grad_reqs_rate', 'seal_of_biliteracy_rate', 'dropout_rate', 'still_enrolled_rate', 'chronicabsenteeismrate', 'eligible_cumulative_enrollment', 'unexcused_absences_percent', 'outofschool_suspension_absences_percent', 'calpads_fall_1_certification_status', 'school_grade_span', 'stu_tch_ratio', 'stu_adm_ratio', 'stu_psv_ratio']

In [59]:
jrep.data_cardinality(df_combined, show_columns=True)

CARDINALITY REPORT

Total columns analyzed: 50

[BINARY COLUMNS]
There are 1 binary columns.
 * Columns: ['yearroundyn']
There are 0 binary with nan.

[CONSTANT/NEAR CONSTANT COLUMNS]
There are 5 constant columns.
 * Columns: ['charter', 'eilcode', 'high_conn', 'low_conn', 'conn_ratio']
There are 11 near-constant columns with >= 95% of values being the same.
 * Columns: ['charter', 'eilcode', 'yearroundyn', 'calpads_fall_1_certification_status', 'school_grade_span', 'stu_tch_ratio', 'stu_adm_ratio', 'stu_psv_ratio', 'high_conn', 'low_conn', 'conn_ratio']

[LOW CARDINALITY CATEGORICAL COLUMNS]
 * There are 8 low cardinality columns with <= 10 unique values.
Columns:
 * charter: 1 unique values
 * eilcode: 1 unique values
 * virtual: 3 unique values
 * magnet: 3 unique values
 * yearroundyn: 2 unique values
 * multilingual: 3 unique values
 * calpads_fall_1_certification_status: 1 unique values
 * school_grade_span: 1 unique values

[HIGH CARDINALITY CATEGORICAL COLUMNS]
 * There are 3 h

In [60]:
df_combined[num_cols].head(10)

,percent__eligible_free_k12,frpm_count_k12,pct_associate,pct_bachelors,pct_bachelors_plus,pct_master,pct_master_plus,pct_doctorate,pct_juris_doctor,pct_no_degree,...,pct_hs_enrollment,pct_senior_cohort,pct_unsafe_gr11,pct_safe_gr11,pct_neutral_gr11,avg_safety_score,high_conn,low_conn,conn_ratio,school_climate_index
0,0.172013,327.0,0.0,0.126316,0.315789,0.126316,0.347368,0.021053,0.0,0.063158,...,1.00000,0.498894,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115
1,0.174389,307.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.00000,0.501264,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115
2,0.262259,935.0,0.0,0.427966,0.148305,0.360169,0.046610,0.016949,0.0,0.000000,...,1.00000,0.496276,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115
3,0.166358,491.0,0.0,0.187919,0.315436,0.161074,0.315436,0.020134,0.0,0.000000,...,1.00000,0.470174,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115
4,0.273810,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.39881,0.196429,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115
5,0.551136,102.0,0.0,0.333333,0.166667,0.333333,0.166667,0.000000,0.0,0.000000,...,1.00000,0.545455,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115
6,0.138382,371.0,0.0,0.410256,0.000000,0.572650,0.000000,0.000000,0.0,0.017094,...,1.00000,0.500397,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115
7,0.144276,366.0,0.0,0.376147,0.000000,0.559633,0.000000,0.036697,0.0,0.027523,...,1.00000,0.497624,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115
8,0.310502,462.0,0.0,0.453333,0.000000,0.466667,0.000000,0.040000,0.0,0.040000,...,1.00000,0.513699,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115
9,0.065449,138.0,0.0,0.389474,0.000000,0.547368,0.000000,0.052632,0.0,0.010526,...,1.00000,0.526129,0.08,0.568,0.351,0.03423,0.333333,0.333333,0.999997,0.517115


The following features can be removed from the dataset due to having only one value:
- charter
- eilcode
- calpads_fall_1_certification_status
- school_grade_span

These high cardinality features can also be removed:
- cdscode
- latitude
- longitude
- county - Though not high cardinality, this column was used only for combining datasets. 

Reviewing the categorical data, most of the columns in categorical are actually numerical values and should be converted.  



In [64]:
# columns to drop
drop_columns = [
    'charter', 'eilcode', 'calpads_fall_1_certification_status',
    'school_grade_span', 'cdscode', 'latitude', 'longitude', 'county'
]

# columns to keep categorical
cat_cols_keep = ['virtual', 'magnet', 'yearroundyn', 'multilingual']

# numerica columns
numeric_cols = [
    # socioeconomic + staff education
    'percent__eligible_free_k12', 'frpm_count_k12',
    'pct_associate', 'pct_bachelors', 'pct_bachelors_plus',
    'pct_master', 'pct_master_plus', 'pct_doctorate', 
    'pct_juris_doctor', 'pct_no_degree',

    # staff experience
    'pct_experienced', 'pct_inexperienced', 
    'pct_first_year', 'pct_second_year',

    # enrollment + graduation
    'cohortstudents', 'eligible_cumulative_enrollment',
    'grade_retention_ratio', 'pct_hs_enrollment', 'pct_senior_cohort',

    # safety + climate
    'pct_unsafe_gr11', 'pct_safe_gr11', 'pct_neutral_gr11',
    'avg_safety_score', 'high_conn', 'low_conn', 
    'conn_ratio', 'school_climate_index',

    # from "categorical" but actually numeric
    'regular_hs_diploma_graduates_rate', 'met_uccsu_grad_reqs_rate', 
    'seal_of_biliteracy_rate', 'dropout_rate', 'still_enrolled_rate',
    'chronicabsenteeismrate', 'unexcused_absences_percent', 
    'outofschool_suspension_absences_percent', 
    'stu_tch_ratio', 'stu_adm_ratio', 'stu_psv_ratio'
]


In [65]:
# drop unwanted columns
df_combined = df_combined.drop(columns=drop_columns, errors = 'ignore')

# convert numeric columns 
df_combined[numeric_cols] = df_combined[numeric_cols].apply(pd.to_numeric, errors = 'coerce')

In [66]:
jrep.data_info(df_combined, show_columns=True)


SHAPE:
There are 1067 rows and 42 columns (0.55 MB).

DUPLICATES:
There are 6 duplicated rows.

COLUMNS/VARIABLES:
Column dType Summary:
 * object: 4
 * float: 38
There are 38 numerical (int/float/bool) variables.
 * Columns: ['cohortstudents', 'regular_hs_diploma_graduates_rate', 'met_uccsu_grad_reqs_rate', 'seal_of_biliteracy_rate', 'dropout_rate', 'still_enrolled_rate', 'chronicabsenteeismrate', 'eligible_cumulative_enrollment', 'unexcused_absences_percent', 'outofschool_suspension_absences_percent', 'percent__eligible_free_k12', 'frpm_count_k12', 'stu_tch_ratio', 'stu_adm_ratio', 'stu_psv_ratio', 'pct_associate', 'pct_bachelors', 'pct_bachelors_plus', 'pct_master', 'pct_master_plus', 'pct_doctorate', 'pct_juris_doctor', 'pct_no_degree', 'pct_experienced', 'pct_inexperienced', 'pct_first_year', 'pct_second_year', 'grade_retention_ratio', 'pct_hs_enrollment', 'pct_senior_cohort', 'pct_unsafe_gr11', 'pct_safe_gr11', 'pct_neutral_gr11', 'avg_safety_score', 'high_conn', 'low_conn', 'co

# Save Combined Dataset

In [67]:
df_combined.to_pickle(data_folder / "ews_dataset.pkl")